<a href="https://colab.research.google.com/github/Tanzir11/Fine_tuned/blob/main/Replicate_Tanzeer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%%capture
!pip install ipython-autotime langchain openai tiktoken faiss-cpu  unstructured transformers pdf2image pdfminer.six replicate --quiet

time: 9.55 s (started: 2023-09-07 16:39:33 +00:00)


In [17]:
import time
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
import openai
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.prompt import PromptTemplate
from langchain.document_loaders import UnstructuredPDFLoader # for loading the pdf
import os
from pprint import pprint
from langchain.llms import Replicate
import replicate

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 7.4 ms (started: 2023-09-07 16:39:43 +00:00)


In [18]:
pdf_path = "/content/Sample_Input_LOI.pdf"
loader = UnstructuredPDFLoader(pdf_path)
input = loader.load_and_split()

time: 240 ms (started: 2023-09-07 16:39:43 +00:00)


In [19]:
input[0].page_content

'Parties Involved: Acquiring Company: - Name: Global Solutions Inc. - Address: 123 Park Avenue, New York, NY 10001 - Contact Person: Mark Johnson - Email: mark.johnson@globalsolutions.com\n\nTarget Company: - Name: Tech Innovators Ltd. - Address: 456 Innovation Road, San Francisco, CA 94107 - Contact Person: Sarah Thompson - Email: sarah.thompson@techinnovators.com\n\nTransaction Details: - Type of Transaction: Acquisition - Proposed Structure: Cash and Stock Purchase - Purchase Price: $250 million - Payment Method: Global Solutions Inc. will pay $150 million in cash and issue $100 million worth of common shares to the shareholders of Tech Innovators Ltd. - Contingency: Approval of the transaction by the respective boards of directors and regulatory authorities.\n\nBackground Information: - Global Solutions Inc. is a multinational technology services company specializing in software development, cloud computing, and cybersecurity solutions. - Tech Innovators Ltd. is a successful startu

time: 8.71 ms (started: 2023-09-07 16:39:43 +00:00)


In [20]:
api = 'Open Ai api key'

time: 509 µs (started: 2023-09-07 16:39:43 +00:00)


In [21]:
embeddings = OpenAIEmbeddings(openai_api_key=api)
index = FAISS.from_documents(input, embeddings)

time: 1 s (started: 2023-09-07 16:39:43 +00:00)


In [22]:
Patent_template = '''

Aim: As an experienced corporate attorney with over 15 years of expertise, you are to draft a Letter of Intent (LOI) based on the provided context and information below. This task involves several steps:

1. Below is a general list of the sections that you need to draft in Letter of Intent (LOI):

  - 1. **Parties Involved:** This section identifies the buyer and seller involved in the transaction, including full names and addresses.

  - 2. **Transaction Overview:** A brief description of the transaction at hand, including the type of transaction (e.g., asset purchase, stock purchase, merger).

  - 3. **Purchase Price and Structure:** This section outlines the proposed purchase price and the structure of the deal. This might include information about how the payment will be made, such as cash, stock, seller financing, or a combination.

  - 4. **Assets and Liabilities:** If it's an asset purchase, the specific assets being bought and liabilities being assumed should be listed here.

  - 5. **Due Diligence and Confidentiality:** An acknowledgement that the buyer will be conducting due diligence and a confidentiality agreement to protect the seller's proprietary information.

  - 6. **Exclusivity Period:** This clause prohibits the seller from soliciting or entertaining offers from other potential buyers for a specified period.

  - 7. **Conditions Precedent:** These are conditions that must be met before a final agreement can be signed. They might include obtaining financing, approval from regulatory bodies, or the completion of due diligence.

  - 8. **Non-binding Nature:** The LOI should make it clear that, apart from specific sections (like confidentiality, exclusivity, etc.), it's not legally binding until a formal, final agreement is signed.

  - 9. **Future Actions:** This outlines the next steps that each party must take to move the transaction forward, such as due diligence process, final agreement drafting etc.

  - 10. **Timeline:** A proposed schedule or timeline for due diligence, negotiations, and completion of the transaction.

  - 11. **Governing Law:** This section indicates the state or country's laws that will govern the LOI and subsequent agreements.

  - 12. **Termination Clause:** This explains the circumstances under which the LOI can be terminated by either party.

  - 13. **Signatures:** Both parties should sign and date the LOI.


2. For each section mentioned, generate a detailed, insightful, and professional analysis based on the provided context and information.

3. for each section, review the draft and refine the content. Generate a revised version that incorporates any improvements identified during the review process.

Remember, the document should be professionally written, well-structured, and contain all necessary legal details. The context and requirements for this task are as follows:

—-----------------------------------------------------------------
context : {context}

Task: {question}
—-----------------------------------------------------------------
'''


time: 978 µs (started: 2023-09-07 16:39:44 +00:00)


In [23]:
PATENT_PROMPT = PromptTemplate(template=Patent_template, input_variables=["context","question" ])

time: 6.35 ms (started: 2023-09-07 16:39:44 +00:00)


In [24]:
retriver = index.as_retriever(search_type='similarity',search_kwargs={"k":15})

time: 1.05 ms (started: 2023-09-07 16:39:44 +00:00)


In [25]:
#pip install replicate
#use the following API token
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = 'replicate_key'

··········
time: 5.28 s (started: 2023-09-07 16:39:46 +00:00)


In [76]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = Replicate(
    # streaming=True, # Uncomment the following line to allow streaming output
    # callbacks=[StreamingStdOutCallbackHandler()],
    model="tanzir11/merge:d89b08de97cee03684b6814330916033ccb4a4586f4493148f7b40d553f4785b",
    input={"temperature": 0.95, "max_new_tokens": 4000, "top_p": 0.95, "repetition_penalty":1.15}
)

time: 1.34 ms (started: 2023-09-07 18:22:58 +00:00)


In [77]:
qa = ConversationalRetrievalChain.from_llm(llm,retriver, return_source_documents=True, max_tokens_limit=8192, combine_docs_chain_kwargs={'prompt': PATENT_PROMPT})

time: 1.56 ms (started: 2023-09-07 18:23:07 +00:00)


In [78]:
# Non-trained 70b model's output
query = "Your response would be taken as it in the input so As a first step of drafting a Letter of Intent (LOI), Your task is to Write '1200' words long content about section-1, i.e... Parties Involved, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Parties Involved:\n'
 '\n'
 'This Letter of Intent (LOI) outlines the proposed transaction between Global '
 'Solutions Inc. (Acquiring Company) and Tech Innovators Ltd. (Target '
 'Company). The parties involved in this transaction are:\n'
 '\n'
 'Global Solutions Inc.:\n'
 '\n'
 '* Name: Global Solutions Inc.\n'
 '* Address: 123 Park Avenue, New York, NY 10001\n'
 '* Contact Person: Mark Johnson\n'
 '* Email: mark.johnson@globalsolutions.com\n'
 '\n'
 'Tech Innovators Ltd.:\n'
 '\n'
 '* Name: Tech Innovators Ltd.\n'
 '* Address: 456 Innovation Road, San Francisco, CA 94107\n'
 '* Contact Person: Sarah Thompson\n'
 '* Email: sarah.thompson@techinnovators.com\n'
 '\n'
 'The Acquiring Company, Global Solutions Inc., is a multinational technology '
 'services company specializing in software development, cloud computing, and '
 'cybersecurity solutions. With a strong presence in the healthcare sector, '
 'Tech Innovators Ltd. is a successful startup specializing in artificial '
 'int

In [104]:
query = f"Your response would be taken as it in the input so do not write anything other than the second step of drafting a Letter of Intent (LOI), Your task is to Write 800 words long content about just only a section-2, i.e... Transaction Overview, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Transaction Overview\n'
 '\n'
 'This Letter of Intent (LOI) outlines the proposed acquisition of Tech '
 'Innovators Ltd. (Target Company) by Global Solutions Inc. (Acquiring '
 'Company). The transaction, valued at $250 million, will be structured as a '
 'cash and stock purchase. The Acquiring Company will pay $150 million in cash '
 'and issue $100 million worth of common shares to the shareholders of the '
 'Target Company.\n'
 '\n'
 'The Acquiring Company, Global Solutions Inc., is a multinational technology '
 'services company specializing in software development, cloud computing, and '
 'cybersecurity solutions. With a strong presence in the healthcare sector, '
 'Tech Innovators Ltd. is a successful startup specializing in artificial '
 'intelligence and machine learning algorithms.\n'
 '\n'
 'The proposed acquisition aims to leverage the complementary strengths and '
 'resources of both companies, creating a powerful synergy that will benefit '
 'both parties and their re

In [103]:
query = f"Your response would be taken as it in the input so do not write anything other than the Third step of drafting a Letter of Intent (LOI), Your task is to Write 800 words long content about just only a section-3, i.e... Purchase Price and Structure, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Purchase Price and Structure\n'
 '\n'
 '3.1. The purchase price for the acquisition of Tech Innovators Ltd. (the '
 '"Company") by Global Solutions Inc. (the "Buyer") shall be Two Hundred Fifty '
 'Million US Dollars ($250,000,000) (the "Purchase Price").\n'
 '\n'
 '3.2. The Purchase Price shall be paid in a combination of cash and stock, '
 'with One Hundred Fifty Million US Dollars ($150,000,000) paid in cash (the '
 '"Cash Payment") and One Hundred Million US Dollars ($100,000,000) paid in '
 'common shares of Global Solutions Inc. (the "Stock Payment").\n'
 '\n'
 '3.3. The Cash Payment shall be made by the Buyer to the Company at the '
 'closing of the transaction (the "Closing").\n'
 '\n'
 "3.4. The Stock Payment shall be made by the Buyer to the Company's "
 'shareholders at the Closing, and shall be based on the average closing price '
 "of Global Solutions Inc.'s common shares on the New York Stock Exchange for "
 'the ten (10) trading days immediately preceding the Closing

In [119]:
query = f"Your response would be taken as it is to genereate official document so do not write anything irrelevant. As a fourth step of drafting a Letter of Intent (LOI), Your task is to Write 800 words long content about just only a section-4, i.e... Assets and Liabilities, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Section 4: Assets and Liabilities\n'
 '\n'
 '4.1. Assets\n'
 '\n'
 'The transaction contemplated by this Letter of Intent (the "Transaction") '
 'will involve the acquisition by Global Solutions Inc. (the "Buyer") of '
 'certain assets of Tech Innovators Ltd. (the "Seller"). The Seller agrees to '
 'sell, transfer, and assign to the Buyer all of its right, title, and '
 'interest in and to the following assets (the "Assets"):\n'
 '\n'
 '4.1.1. Intellectual Property\n'
 '\n'
 'The Seller agrees to sell, transfer, and assign to the Buyer all of its '
 'right, title, and interest in and to the intellectual property rights owned '
 'or licensed by the Seller, including but not limited to:\n'
 '\n'
 '* Patents and patent applications\n'
 '* Trademarks and service marks\n'
 '* Copyrights\n'
 '* Trade secrets and confidential information\n'
 '\n'
 'The Seller represents and warrants that it has the right and authority to '
 'sell, transfer, and assign the intellectual property rights to t

In [118]:
query = f"Your response would be taken as it is to genereate official document so do not write anything irrelevant. As a fifth step of drafting a Letter of Intent (LOI), Your task is to Write 800 words long content about just only a section-5, i.e... Due Diligence and Confidentiality, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Section 5: Due Diligence and Confidentiality\n'
 '\n'
 '5.1 Due Diligence\n'
 '\n'
 'The parties acknowledge that due diligence is a critical aspect of the '
 'transaction and agree to conduct thorough due diligence investigations to '
 'assess the financial, legal, and operational aspects of Tech Innovators Ltd. '
 '(the "Company"). The Company agrees to provide access to audited financial '
 'statements, customer contracts, intellectual property records, and relevant '
 'regulatory compliance documentation. The parties agree to work together in '
 'good faith to ensure that the due diligence process is completed in a timely '
 'and efficient manner.\n'
 '\n'
 '5.2 Confidentiality\n'
 '\n'
 'The parties acknowledge that the information exchanged during the '
 'negotiation process is confidential and agree to maintain strict '
 'confidentiality and safeguard all proprietary and sensitive information. The '
 'parties agree to keep all information and documents exchanged during the d

In [121]:
query = f"Your response would be taken as it is to genereate official document so do not write anything irrelevant. As a sixth step of drafting a Letter of Intent (LOI), Your task is to Write 800 words long content about just only a section-6, i.e... Exclusivity Period, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Section 6: Exclusivity Period\n'
 '\n'
 '6.1. The parties agree that, for a period of sixty (60) days from the date '
 'of this Letter of Intent (the "Exclusivity Period"), neither party shall, '
 'directly or indirectly, solicit, initiate, encourage, or engage in any '
 'discussions or negotiations with any third party regarding any transaction '
 'that could reasonably be expected to compete with the transaction '
 'contemplated by this Letter of Intent.\n'
 '\n'
 '6.2. During the Exclusivity Period, each party shall notify the other '
 'promptly in writing of any approach or proposal received from a third party '
 'in respect of a competing transaction.\n'
 '\n'
 '6.3. The parties acknowledge and agree that the Exclusivity Period is a '
 'material inducement to the other party to enter into this Letter of Intent '
 'and that any breach of this Section 6 by a party shall be a material breach '
 'of this Letter of Intent.\n'
 '\n'
 '6.4. Each party agrees to use its best efforts t

In [99]:
query = f"Your response would be taken as it in the input so do not write anything other than the seventh step of drafting a Letter of Intent (LOI), Your task is to Write 800 words long content about just only a section-7, i.e... Conditions Precedent, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Conditions Precedent:\n'
 '\n'
 '7.1. In order to facilitate the successful completion of the transaction '
 'contemplated by this Letter of Intent (LOI), both parties agree to undertake '
 'certain actions and fulfill specific conditions precedent. These conditions '
 'are essential to ensure that the transaction is consummated in a timely and '
 'efficient manner, and that all relevant regulatory approvals are obtained.\n'
 '\n'
 '7.2. The following conditions precedent must be met before the parties can '
 'proceed with the transaction:\n'
 '\n'
 '7.2.1. Regulatory Approvals: Both parties shall use their commercially '
 'reasonable efforts to obtain all necessary regulatory approvals from the '
 'Federal Trade Commission (FTC) and any other relevant authorities. The '
 'parties acknowledge that the transaction is subject to obtaining these '
 'approvals and that the failure to obtain them may result in the termination '
 'of the LOI.\n'
 '\n'
 '7.2.2. Due Diligence: Both parties

In [114]:
query = f"Your response would be taken as it is to genereate official document so do not write anything irrelevant eigth step of drafting a Letter of Intent (LOI), Your task is to Write 800 words long content about just only a section-8, i.e... Non-binding Nature, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Section 8: Non-Binding Nature\n'
 '\n'
 '8.1 The parties acknowledge and agree that, except for the provisions of '
 'this Letter of Intent (LOI) related to confidentiality, exclusivity, and '
 'termination, this document is not intended to be, and shall not be construed '
 'as, a legally binding agreement.\n'
 '\n'
 '8.2 The parties expressly acknowledge and agree that the terms and '
 'conditions of this LOI are non-binding and are not intended to create any '
 'legally enforceable rights or obligations.\n'
 '\n'
 '8.3 The parties further acknowledge and agree that the non-binding nature of '
 'this LOI shall not preclude either party from pursuing any legal remedies '
 'available to it in the event of a breach of any of the provisions of this '
 'LOI.\n'
 '\n'
 '8.4 The parties agree that the non-binding nature of this LOI shall not be '
 'construed as a waiver of any rights or remedies that either party may have '
 'under applicable law.\n'
 '\n'
 '8.5 The parties acknowledge a

In [91]:
query = f"Your response would be taken as it in the input so do not write anything other than the ninth step of drafting a Letter of Intent (LOI), Your task is to Write 800 words long content about just only a section-9, i.e... Future Actions, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Future Actions\n'
 '\n'
 '9.1. Due Diligence\n'
 '\n'
 'Upon execution of this Letter of Intent, both parties will proceed with '
 'thorough due diligence investigations to assess the financial, legal, and '
 'operational aspects of Tech Innovators Ltd. The due diligence process will '
 'include, but not be limited to, the review of audited financial statements, '
 'customer contracts, intellectual property records, and relevant regulatory '
 'compliance documentation. Tech Innovators Ltd. will provide access to all '
 'necessary information and documentation to Global Solutions Inc., and both '
 'parties will work collaboratively to ensure a comprehensive and efficient '
 'due diligence process.\n'
 '\n'
 '9.2. Definitive Agreements\n'
 '\n'
 'Following the completion of due diligence, the parties will negotiate and '
 'execute definitive agreements, including a stock purchase agreement, asset '
 'purchase agreement, and any other necessary documentation. These agreements '
 'will

In [113]:
query = f"Your response would be taken as it is to genereate official document so do not write anything irrelevant. Your task is to Write 800 words long content about just only a section-10, i.e... Timeline, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Section 10: Timeline\n'
 '\n'
 '10.1 Overview\n'
 '\n'
 'The following timeline outlines the key milestones and deadlines for the '
 'transaction contemplated by this Letter of Intent (LOI). The parties agree '
 'to work diligently towards completing the transaction within the timeframe '
 'set forth below, subject to the satisfaction of all conditions precedent.\n'
 '\n'
 '10.2 Negotiation and Execution of Definitive Agreements\n'
 '\n'
 'The parties will negotiate and execute definitive agreements, including a '
 'stock purchase agreement, asset purchase agreement, or merger agreement, as '
 'applicable, within 60 days from the signing of the LOI. The definitive '
 'agreements will contain customary representations, warranties, covenants, '
 'and conditions precedent to closing.\n'
 '\n'
 '10.3 Due Diligence Investigations\n'
 '\n'
 'Both parties will conduct thorough due diligence investigations to assess '
 'the financial, legal, and operational aspects of the target company. T

In [95]:
query = f"Your response would be taken as it in the input so do not write anything other than the eleventh step of drafting a Letter of Intent (LOI), Your task is to Write 800 words long content about just only a section-11, i.e... Governing Law, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Governing Law:\n'
 '\n'
 '11.1. This Letter of Intent (LOI) and any subsequent agreements shall be '
 'governed by and interpreted in accordance with the laws of the State of '
 'Delaware, USA.\n'
 '\n'
 '11.2. The parties agree that any disputes arising out of or in connection '
 'with this LOI will be resolved through binding arbitration in accordance '
 'with the rules of the American Arbitration Association.\n'
 '\n'
 '11.3. The arbitration tribunal shall consist of three arbitrators, with each '
 'party appointing one arbitrator and the third arbitrator being selected by '
 'the two party-appointed arbitrators.\n'
 '\n'
 '11.4. The arbitration proceedings shall be conducted in the English language '
 'and shall be held in the City of New York, State of New York, USA.\n'
 '\n'
 '11.5. The arbitration award shall be final and binding on the parties, and '
 'the parties agree to abide by and implement the arbitration award without '
 'delay.\n'
 '\n'
 '11.6. The parties agree tha

In [96]:
query = f"Your response would be taken as it in the input so do not write anything other than the twelfth step of drafting a Letter of Intent (LOI), Your task is to Write 800 words long content about just only a section-12, i.e... Termination Clause, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Termination Clause\n'
 '\n'
 '12.1 This Letter of Intent (LOI) may be terminated by either party upon '
 'written notice to the other party in the event of a material breach of this '
 'LOI by the other party.\n'
 '\n'
 '12.2 For purposes of this LOI, a material breach shall include, but not be '
 'limited to, the failure to meet the conditions precedent set forth in '
 'Section 6 above.\n'
 '\n'
 '12.3 In the event of a material breach by one party, the other party shall '
 'have the right to terminate this LOI and pursue any and all remedies '
 'available at law or in equity.\n'
 '\n'
 '12.4 This LOI may also be terminated by either party if the transaction '
 'contemplated by this LOI is unable to secure the necessary regulatory '
 'approvals within a specified timeframe.\n'
 '\n'
 '12.5 In the event of termination of this LOI, all obligations and '
 'commitments of the parties hereunder shall immediately cease and terminate, '
 'and each party shall be released from all liabili

In [123]:
query = f"Your response would be taken as it is to genereate official document so do not write anything irrelevant. Your task is to Write 800 words long content about just only a section-13, i.e... Signatures, based on the context and memory available."
result = qa({"question":query,"chat_history":[]})
pprint(result["answer"])

('  Section 13: Signatures\n'
 '\n'
 '13.1. The LOI shall be executed by the authorized representatives of both '
 'parties, Global Solutions Inc. and Tech Innovators Ltd.\n'
 '\n'
 '13.2. The signing of the LOI indicates that both parties have read, '
 'understood, and agree to be bound by the terms and conditions outlined in '
 'the document.\n'
 '\n'
 '13.3. The authorized representatives of Global Solutions Inc. and Tech '
 'Innovators Ltd. shall sign the LOI in the presence of witnesses, and the '
 'signatures shall be notarized.\n'
 '\n'
 '13.4. The LOI shall be executed in duplicate originals, with each party '
 'retaining one original copy.\n'
 '\n'
 '13.5. The signatures of the authorized representatives shall be affixed to '
 'the LOI after the parties have negotiated and agreed upon all the terms and '
 'conditions.\n'
 '\n'
 '13.6. The LOI shall come into effect on the date of its execution and shall '
 'remain in force until the earlier of (i) the completion of the transac